In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn import tree, ensemble, metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
smoking_df = pd.read_csv('/content/drive/MyDrive/smoking.csv')
smoking_df.head(10)

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar,smoking
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,12.9,1.0,0.7,18.0,19.0,27.0,Y,0,Y,0
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,12.7,1.0,0.6,22.0,19.0,18.0,Y,0,Y,0
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,15.8,1.0,1.0,21.0,16.0,22.0,Y,0,N,1
3,3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,...,14.7,1.0,1.0,19.0,26.0,18.0,Y,0,Y,0
4,4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,...,12.5,1.0,0.6,16.0,14.0,22.0,Y,0,N,0
5,5,M,30,180,75,85.0,1.2,1.2,1.0,1.0,...,16.2,1.0,1.2,18.0,27.0,33.0,Y,0,Y,0
6,6,M,40,160,60,85.5,1.0,1.0,1.0,1.0,...,17.0,1.0,0.7,21.0,27.0,39.0,Y,1,Y,1
7,7,M,45,165,90,96.0,1.2,1.0,1.0,1.0,...,15.0,1.0,1.3,38.0,71.0,111.0,Y,0,Y,0
8,9,F,50,150,60,85.0,0.7,0.8,1.0,1.0,...,13.7,1.0,0.8,31.0,31.0,14.0,Y,0,N,0
9,10,M,45,175,75,89.0,1.0,1.0,1.0,1.0,...,16.0,1.0,0.8,26.0,24.0,63.0,Y,0,N,0


In [4]:
smoking_df.dtypes

ID                       int64
gender                  object
age                      int64
height(cm)               int64
weight(kg)               int64
waist(cm)              float64
eyesight(left)         float64
eyesight(right)        float64
hearing(left)          float64
hearing(right)         float64
systolic               float64
relaxation             float64
fasting blood sugar    float64
Cholesterol            float64
triglyceride           float64
HDL                    float64
LDL                    float64
hemoglobin             float64
Urine protein          float64
serum creatinine       float64
AST                    float64
ALT                    float64
Gtp                    float64
oral                    object
dental caries            int64
tartar                  object
smoking                  int64
dtype: object

In [5]:
# Generate our categorical variable lists
smoking_cat = smoking_df.dtypes[smoking_df.dtypes == "object"].index.tolist()

In [6]:
# Check the number of unique values in each column
smoking_df[smoking_cat].nunique()

gender    2
oral      1
tartar    2
dtype: int64

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(smoking_df[smoking_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(smoking_cat)
encode_df.head()

,gender_F,gender_M,oral_Y,tartar_N,tartar_Y
0,1.0,0.0,1.0,0.0,1.0
1,1.0,0.0,1.0,0.0,1.0
2,0.0,1.0,1.0,1.0,0.0
3,0.0,1.0,1.0,0.0,1.0
4,1.0,0.0,1.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
smoking_df = smoking_df.merge(encode_df,left_index=True, right_index=True)
smoking_df = smoking_df.drop(smoking_cat,1)
smoking_df.head()

<ipython-input-10-07605a05507c>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  smoking_df = smoking_df.drop(smoking_cat,1)


,ID,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,AST,ALT,Gtp,dental caries,smoking,gender_F,gender_M,oral_Y,tartar_N,tartar_Y
0,0,40,155,60,81.3,1.2,1.0,1.0,1.0,114.0,...,18.0,19.0,27.0,0,0,1.0,0.0,1.0,0.0,1.0
1,1,40,160,60,81.0,0.8,0.6,1.0,1.0,119.0,...,22.0,19.0,18.0,0,0,1.0,0.0,1.0,0.0,1.0
2,2,55,170,60,80.0,0.8,0.8,1.0,1.0,138.0,...,21.0,16.0,22.0,0,1,0.0,1.0,1.0,1.0,0.0
3,3,40,165,70,88.0,1.5,1.5,1.0,1.0,100.0,...,19.0,26.0,18.0,0,0,0.0,1.0,1.0,0.0,1.0
4,4,40,155,60,86.0,1.0,1.0,1.0,1.0,120.0,...,16.0,14.0,22.0,0,0,1.0,0.0,1.0,1.0,0.0


In [11]:
smoking_df.dtypes

ID                       int64
age                      int64
height(cm)               int64
weight(kg)               int64
waist(cm)              float64
eyesight(left)         float64
eyesight(right)        float64
hearing(left)          float64
hearing(right)         float64
systolic               float64
relaxation             float64
fasting blood sugar    float64
Cholesterol            float64
triglyceride           float64
HDL                    float64
LDL                    float64
hemoglobin             float64
Urine protein          float64
serum creatinine       float64
AST                    float64
ALT                    float64
Gtp                    float64
dental caries            int64
smoking                  int64
gender_F               float64
gender_M               float64
oral_Y                 float64
tartar_N               float64
tartar_Y               float64
dtype: object

In [12]:
# Split our preprocessed data into our features and target arrays
y = smoking_df["smoking"].values
X = smoking_df.drop(["smoking"],axis = 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(41769, 28)
(13923, 28)
(41769,)
(13923,)


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [32]:
# Random Forest test
random_forest_classifier = ensemble.RandomForestClassifier(n_estimators=50, max_depth=15, min_samples_split=5, min_samples_leaf=45).fit(X, y)
print(f"Accuracy for selected random fores model: {metrics.accuracy_score(y, random_forest_classifier.predict(X))}")

Accuracy for selected random fores model: 0.8040472599296129


In [18]:
# tree classifier test
tree_classifier = tree.DecisionTreeClassifier(max_depth=15, min_samples_split=5, min_samples_leaf=100).fit(X, y)
print(f"Accuracy for selected tree model: {metrics.accuracy_score(y, tree_classifier.predict(X))}")

Accuracy for selected tree model: 0.7747252747252747


In [27]:
# Making predictions using the testing data
predictions = tree_classifier.predict(X_test_scaled)
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [29]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,6711,2061
Actual 1,2078,3073


              precision    recall  f1-score   support

           0       0.76      0.77      0.76      8772
           1       0.60      0.60      0.60      5151

    accuracy                           0.70     13923
   macro avg       0.68      0.68      0.68     13923
weighted avg       0.70      0.70      0.70     13923



In [16]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
number_input_features

28

In [19]:
hidden_nodes_layer1 =  60
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                1740      
                                                                 
 dense_1 (Dense)             (None, 30)                1830      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3,601
Trainable params: 3,601
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1306/1306 [==============================] - 6s 3ms/step - loss: 0.4800 - accuracy: 0.7417
Epoch 2/100
1306/1306 [==============================] - 2s 2ms/step - loss: 0.4576 - accuracy: 0.7598
Epoch 3/100
1306/1306 [==============================] - 3s 2ms/step - loss: 0.4521 - accuracy: 0.7652
Epoch 4/100
1306/1306 [==============================] - 4s 3ms/step - loss: 0.4497 - accuracy: 0.7667
Epoch 5/100
1306/1306 [==============================] - 3s 2ms/step - loss: 0.4472 - accuracy: 0.7696
Epoch 6/100
1306/1306 [==============================] - 3s 3ms/step - loss: 0.4458 - accuracy: 0.7719
Epoch 7/100
1306/1306 [==============================] - 2s 2ms/step - loss: 0.4437 - accuracy: 0.7743
Epoch 8/100
1306/1306 [==============================] - 2s 2ms/step - loss: 0.4419 - accuracy: 0.7751
Epoch 9/100
1306/1306 [==============================] - 2s 2ms/step - loss: 0.4404 - accuracy: 0.7749
Epoch 10/100
1306/1306 [==============================] - 3s 2ms/step - l

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

436/436 - 1s - loss: 0.5086 - accuracy: 0.7521 - 725ms/epoch - 2ms/step
Loss: 0.5085675716400146, Accuracy: 0.7520649433135986
